<a href="https://colab.research.google.com/github/shuyu-d/IDL_TP/blob/master/tp5_idl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP 5 (Exercice 6.3) - Perceptron multicouches (MLP)

Dans ce TP, nous implémentons les fonctions pour le calcul du passe avant et du passe arrière pour entraîner un MLP construit avec Numpy.

Les données d'observation appartiennent au jeu de données Iris, déjà disponible via scikit-learn (`sklearn datasets`, voir "Préparation du jeu de données"). Plus d'information sur ce jeu de données : https://scikit-learn.org/1.4/auto_examples/datasets/plot_iris_dataset.html,   https://archive.ics.uci.edu/dataset/53/iris. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
pd.set_option('display.expand_frame_repr', False)
from sklearn import datasets
from sklearn.model_selection import train_test_split

### Préparation du jeu de données 
Ce jeu de données contient trois espèces différentes d’iris (Setosa, Versicolour et Virginica), caractérisées par la longueur et la largeur des sépales et des pétales. 
Les données sont stockées dans un tableau NumPy de dimension 150 x 4.

Les lignes correspondent aux observations (échantillons) et les colonnes représentent respectivement : longueur du sépale, largeur du sépale, longueur du pétale et largeur du pétale.

In [ ]:
iris = datasets.load_iris()
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_df['label'] = [iris.target_names[y] for y in iris.target]
iris_df.sample(frac=1).head(10)

### Classification binaire (Setosa versus non-Setosa) 
Avec le jeu de données Iris, on considère le problème de classification binaire qui vise à distinguer l'espèce Setosa des deux autres$^*$. Ainsi, la classe binaire $y_i$ de chaque échantillon est définie par 
 * $y_i = 0$ si le label de l'échantillon est 'setosa'. On appelle cette classe 'setosa'
 * $y_1 = 1$ sinon. On appelle cette classe 'non-setosa'. 


In [ ]:
iris_data = iris.data.copy() 
bin_target = iris.target.copy()
bin_target_names = iris.target_names.copy()
bin_target[ bin_target != 0] = 1
bin_target_names[bin_target_names != bin_target_names[0]] = 'non-setosa' 

In [ ]:
plt.rcParams.update({'font.size': 18})

fig, ax = plt.subplots()
scatter = ax.scatter(iris_data[:, 0], iris_data[:, 1], c=bin_target)
ax.set(xlabel=iris.feature_names[0], ylabel=iris.feature_names[1])
_ = ax.legend(scatter.legend_elements()[0], bin_target_names, loc="lower right", 
    bbox_to_anchor=(0.15,1.02,1,0.2),ncol=3)
fig, ax = plt.subplots()
scatter = ax.scatter(iris_data[:, 0], iris_data[:, 2], c=bin_target)
ax.set(xlabel=iris.feature_names[0], ylabel=iris.feature_names[2])
_ = ax.legend(scatter.legend_elements()[0], bin_target_names, loc="lower right", 
    bbox_to_anchor=(0.15,1.02,1,0.2),ncol=3)
fig, ax = plt.subplots()
scatter = ax.scatter(iris_data[:, 0], iris_data[:, 3], c=bin_target)
ax.set(xlabel=iris.feature_names[0], ylabel=iris.feature_names[3])
_ = ax.legend(scatter.legend_elements()[0], bin_target_names, loc="lower right", 
    bbox_to_anchor=(0.15,1.02,1,0.2),ncol=3)
## 
fig, ax = plt.subplots()
scatter = ax.scatter(iris_data[:, 1], iris_data[:, 2], c=bin_target)
ax.set(xlabel=iris.feature_names[1], ylabel=iris.feature_names[2])
_ = ax.legend(scatter.legend_elements()[0], bin_target_names, loc="lower right", 
    bbox_to_anchor=(0.15,1.02,1,0.2),ncol=3)
fig, ax = plt.subplots()
scatter = ax.scatter(iris_data[:, 1], iris_data[:, 3], c=bin_target)
ax.set(xlabel=iris.feature_names[1], ylabel=iris.feature_names[3])
_ = ax.legend(scatter.legend_elements()[0], bin_target_names, loc="lower right", 
    bbox_to_anchor=(0.15,1.02,1,0.2),ncol=3)
fig, ax = plt.subplots()
scatter = ax.scatter(iris_data[:, 2], iris_data[:, 3], c=bin_target)
ax.set(xlabel=iris.feature_names[2], ylabel=iris.feature_names[3])
_ = ax.legend(scatter.legend_elements()[0], bin_target_names, loc="lower right", 
    bbox_to_anchor=(0.15,1.02,1,0.2),ncol=3)


On définit $X$ le vecteur aléatoire associé aux quatres charactéristiques dans le jeu de données Iris. Maintenant l'ensemble $\mathcal{D} = (x_i, y_i)_{i=1}^n$ avec $x_i\in\mathbb{R}^4$ et $y_i \in \{0,1\}$. 
 

In [ ]:
X = np.array( iris_data ) 
y = bin_target # 0 means class 'setosa', 1 means class 'non-setosa' (the other two types)

On divise le jeu de données en deux parties : une partie pour l'entraînement du MLP, et une partie complémentaire pour évaluer la prédiction par le MLP entraîné. 

In [ ]:
print('Check out a few lines of the matrix X (shape %d x %d)'%(X.shape) )
print( pd.DataFrame(X).head() )

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

n, p = X_train.shape

print('A few rows of Xtrain (shape %d x %d) : \n' %(n,p), pd.DataFrame(X_train).head() ) 
print('A few rows of y_train (shape %d x 1): \n'%y_train.shape[0], pd.DataFrame(y_train).head() ) 

###  Construction et initialisation d'un MLP 

**Parametres du MLP** 

* Entrées : $p=4$ 
* 2 couches cachées : $\sigma_1=$ReLU, $\sigma_2=$ReLU
    * couche cachée 1: 3 neurones
    * couche cachée 2: 4 neurones
* Sortie : 
    * $\sigma_3$=sigmoid 
    * $y=f_{\theta}(x) \in [0,1]$ 
  
**Risque empirique** 

On définit le risque empirique suivant comme l'objectif de l'entraînement : 
$$ R(\theta) = \frac{1}{2}\sum_{i=1}^n ( f_{\theta}(x_i) - y_i )^2.$$


In [ ]:
# Parametres du MLP (2 couches cachées : sigma_1=ReLU, sigma_2=ReLU, sortie sigma_3=sigmoid)
#  - couche cachée 1: 3 neurones
#  - couche cachée 2: 4 neurones
np.random.seed(0)

W1 = np.random.randn(p, 3) * 0.1
b1 = np.zeros((1, 3))

W2 = np.random.randn(3, 4) * 0.1
b2 = np.zeros((1, 4))

W3 = np.random.randn(4, 1) * 0.1
b3 = np.zeros((1, 1))

# Fonctions d'activation
def relu(z):
    return np.maximum(0, z)

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# Dérivée de la fonction ReLU 
def relu_derivative(z):
    return (z > 0).astype(float)


#### Question (a) : passe avant

A l'aide de l'Exercice 6.2 (i), compléter la fonction `passe_avant()` pour l'évaluation de la fonction de prédiction $f_{\theta}(\cdot)$ sur un ensemble de données $X$, étant donné le paramètre $\theta=(W^{(i)}, b^{(i)})_{i=1,2,3}$ du MLP. 



In [ ]:
# Passe avant 
def passe_avant(X, W1, W2, W3, b1, b2, b3): 
    # ---- A COMPLÉTER ------
    Z1 = 
    A1 = 
    Z2 = 
    A2 = 
    Z3 =  
    Y  = 
    # -----------------------
    return (Z1, A1, Z2, A2, Z3, Y)

#### Question (b) : passe arrière 
A l'aide des équations pour le calcul des dérivées partielles $\frac{\partial R}{\partial W^{(\ell)}_{ij}}$, compléter les lignes pour le passe arrière dans le bloc de l'algorithme GD suivant. 

In [ ]:
# Entrainement par rétropropagation

# Algorithme : décente de gradient avec stepsize fixe 
#   lr (pour learning rate): un stepsize fixe   
#   niters_max             : nombre maximal d'iterations 
lr = 0.05
niters_max = 5000

for niter in range(niters_max):
    # Passe avant 
    (Z1, A1, Z2, A2, Z3, Y_hat) = passe_avant(X_train, W1, W2, W3, b1, b2, b3)
    
    # Observer la fonction de perte par la cross-entropie 
    loss = - np.mean(y_train * np.log(Y_hat + 1e-8) + (1 - y_train) * np.log(1 - Y_hat + 1e-8) )

    # Retropropagation
    
    #-------A COMPLÉTER -----------
    dZ3 =  # delta^3 := dR/dz^{(3)} de l'Eq (1)
    dW3 =  # dR/dW^{(3)}
    db3 =  # dR/db^{(3)}

    dA2 =  # nom suggéré pour une dérivée partielle intermédiare 
    dZ2 =  # d(delta^3)/dz^{(2)} de l'Eq (2)
    dW2 = 
    db2 = 

    dA1 =  # nom suggéré pour une dérivée partielle intermédiare 
    dZ1 = 
    dW1 = 
    db1 = 
    #--------------------------------

    # Décente de gradient 
    W3 -= lr * dW3
    b3 -= lr * db3

    W2 -= lr * dW2
    b2 -= lr * db2

    W1 -= lr * dW1
    b1 -= lr * db1

    if niter % 1000 == 0:
        print(f"niter {niter}, Loss: {loss:.4f}")

In [ ]:
# Evaluation du MLP 
outputs = passe_avant(X_test, W1, W2, W3, b1, b2, b3)
Y_hat_test = outputs[-1]

predictions = (Y_hat_test > 0.5).astype(int)
accuracy = np.mean(predictions == y_test)

print("Test accuracy:", accuracy)